## Using Twitter official API

**Notes**   
Great resource
https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/ipynb/Chapter%201%20-%20Mining%20Twitter.ipynb

Go to twitter dev to get an API key.
https://dev.twitter.com/apps

Must add a phone number. 

In [4]:
import twitter
from python.API_KEYS_TWITTER import *  # tell students not to put credentials on github
auth = twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

Obtaining a search result. This is a bit more complicated than requesting n=1000 tweets. Some issues:

1. We need to parse the information returned, as it isn't in a neat Dataframe for us to begin analyzing.

2. We need to make multiple requests to get an amount good for analysis. The limit for the Twitter API is 100 tweets per request and 180 requests per 15 minutes, which gives us 18,000 for 15 minutes.

3. We need to make sure we're not getting duplicate tweets. Since tweets and tweet counts are always changing, we do this with tweet IDs. For the guide I used, see: https://dev.twitter.com/rest/public/timelines

### Let's start by grabbing the first result.

In [90]:
# https://dev.twitter.com/rest/reference/get/search/tweets
# general queries: https://dev.twitter.com/rest/public/search

SEARCH = "Super Bowl"  # note that no tweets older than 1 week will be found
search_results = twitter_api.search.tweets(q=SEARCH, count=100)

Now let's investigate the structure of this object.

In [91]:
type(search_results)

twitter.api.TwitterDictResponse

In [92]:
search_results.keys()

dict_keys(['statuses', 'search_metadata'])

Since we want to look at the actual text of the tweets, let's grab the values instead.

In [120]:
results = search_results.values()
results = list(results)  # conver the dict_view object to a indexable list

Check out the results (too long to print)

In [121]:
# results

Looks like we have the results from 100 tweets. Now we can extract the text. Some experimentation gives us the structure of the `results` list.

`results[0: search data or 1: metadata][tweet #][information type]`

**Let's extract the text and turn it into a dataframe.**

In [157]:
# Figure out what keys are available in the python dictionary
results[0][0].keys()

dict_keys(['in_reply_to_user_id', 'geo', 'metadata', 'user', 'retweeted', 'place', 'extended_entities', 'source', 'id_str', 'possibly_sensitive', 'in_reply_to_user_id_str', 'id', 'truncated', 'retweet_count', 'contributors', 'coordinates', 'in_reply_to_status_id', 'retweeted_status', 'in_reply_to_screen_name', 'lang', 'is_quote_status', 'in_reply_to_status_id_str', 'entities', 'text', 'favorited', 'favorite_count', 'created_at'])

In [149]:
results[0][0]['text']

"RT @GMA: Retweet if you're picking the @AtlantaFalcons to win the Super Bowl! \n\n#SB51 #Falcons #RiseUp https://t.co/Wvm8BlrFbC"

In [134]:
results[0]

list

In [108]:
tweets = []
for i in range(100):
    tweets.append(results[0][i]['text'])
tweets[0:2]  # take a look at the first 2 tweets

["RT @GMA: Retweet if you're picking the @AtlantaFalcons to win the Super Bowl! \n\n#SB51 #Falcons #RiseUp https://t.co/Wvm8BlrFbC",
 "Ain't shit else to do So I might as well watch this corny shit..oh yeah #GoFalcons #RiseUp cause I hate New England — watching Super Bowl LI"]

In [119]:
import pandas as pd
dat = pd.DataFrame(pd.Series(tweets), columns=['tweet_text'])
dat.head()

,tweet_text
0,RT @GMA: Retweet if you're picking the @Atlant...
1,Ain't shit else to do So I might as well watch...
2,RT @SoDamnTrue: I have yet to see a Super Bowl...
3,#SuperBowl livestream 👉 https://t.co/CTdkHRdcDd
4,RT @SportsCenter: RT if you think the Falcons ...


Awesome. We got and processed our data. Now let's grab a lot more tweets. 1000 tweets, or 10 requests, should be enough to play with.